In [11]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp /home/Personal_Projects/Databricks/work/.kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.0 MB/s eta 0:00:00


In [12]:
#Download the dataset with Kaggle CLI
!kaggle datasets download -d blastchar/telco-customer-churn -p /home/Personal_Projects/Databricks/work/datasets
!unzip -o /home/Personal_Projects/Databricks/work/datasets/telco-customer-churn.zip -d /home/Personal_Projects/Databricks/work/datasets


Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
telco-customer-churn.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /home/Personal_Projects/Databricks/work/datasets/telco-customer-churn.zip
  inflating: /home/Personal_Projects/Databricks/work/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [1]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except:
    pass

# Create Spark session
spark = SparkSession.builder \
    .appName("TelcoCustomerChurn") \
    .master("local[*]") \
    .getOrCreate()

datapath = "/home/Personal_Projects/Databricks/work/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv"

# Load dataset
df = spark.read.csv(
    datapath,
    header=True,
    inferSchema=True
)

# Inspect schema and data
df.printSchema()
df.show(5, truncate=False)


root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)

+----------+------+-------------+-------+----------+------+------------+---------

In [4]:
#LOAD DATA -- BRONZE LAYER


# Load raw data (Bronze)
bronze_df = spark.read.csv(datapath, header=True, inferSchema=True)

bronze_df.show(5)

#VIEW IN TABLES
bronze_pd = bronze_df.limit(20).toPandas()
bronze_pd.head(10)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


# SILVER LAYER
### Convert data types,Handle missing values,Trim spaces / cleanup,Save in Parquet
### Deduplicates customerID,Adds derived column IsSenior,Performs simple data quality checks (tenure >= 0),Keeps Yes/No columns as strings (no conversion to 1/0)

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, when

# Start Spark session
spark = SparkSession.builder \
    .appName("TelcoSilverLayerStrings") \
    .getOrCreate()

# ==========================
# Step 1: Read Bronze CSV
# ==========================
bronze_df = spark.read.csv(
    "/home/Personal_Projects/Databricks/work/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv",
    header=True,
    inferSchema=True
)

# ==========================
# Step 2: Silver transformations
# ==========================
silver_df = (
    bronze_df
    # Trim all string columns
    .withColumn("customerID", trim(col("customerID")))
    .withColumn("gender", trim(col("gender")))
    .withColumn("Partner", trim(col("Partner")))
    .withColumn("Dependents", trim(col("Dependents")))
    .withColumn("PhoneService", trim(col("PhoneService")))
    .withColumn("MultipleLines", trim(col("MultipleLines")))
    .withColumn("InternetService", trim(col("InternetService")))
    .withColumn("OnlineSecurity", trim(col("OnlineSecurity")))
    .withColumn("OnlineBackup", trim(col("OnlineBackup")))
    .withColumn("DeviceProtection", trim(col("DeviceProtection")))
    .withColumn("TechSupport", trim(col("TechSupport")))
    .withColumn("StreamingTV", trim(col("StreamingTV")))
    .withColumn("StreamingMovies", trim(col("StreamingMovies")))
    .withColumn("Contract", trim(col("Contract")))
    .withColumn("PaperlessBilling", trim(col("PaperlessBilling")))
    .withColumn("PaymentMethod", trim(col("PaymentMethod")))
    
    # Convert data types
    .withColumn("SeniorCitizen", col("SeniorCitizen").cast("int"))
    .withColumn("tenure", col("tenure").cast("int"))
    .withColumn("MonthlyCharges", col("MonthlyCharges").cast("double"))
    .withColumn("TotalCharges", when(trim(col("TotalCharges")) == "", None)
                .otherwise(col("TotalCharges").cast("double")))

    # Handle missing critical values
    .dropna(subset=["customerID", "tenure", "MonthlyCharges", "TotalCharges"])

    # Derived column
    .withColumn("IsSenior", when(col("SeniorCitizen") == 1, True).otherwise(False))

    # Data quality checks
    .filter(col("tenure") >= 0)

    # Deduplicate
    .dropDuplicates(["customerID"])
)

# ==========================
# Step 3: Save Silver layer as Parquet
# ==========================
silver_path = "/home/Personal_Projects/Databricks/work/data/silver/telco_churn"
spark.catalog.clearCache()
silver_df.write.mode("overwrite").parquet(silver_path)

# ==========================
# Step 4: Show sample data
# ==========================
silver_df.show(10, truncate=False)
silver_df.printSchema()

#VIEW IN TABLES FORMAT
silver_db=silver_df.limit(20).toPandas()
silver_db.head(10)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+-------------------------+--------------+------------+-----+--------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract      |PaperlessBilling|PaymentMethod            |MonthlyCharges|TotalCharges|Churn|IsSenior|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+-------------------------+--------------+------------+-----+--------+
|0002-ORFBO|Female|0            |Yes    |Yes       |9     |Yes         |No           |DSL            |No            |Yes         |No 

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,IsSenior
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,DSL,No,...,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30,No,False
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,DSL,No,...,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40,No,False
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85,Yes,False
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85,Yes,True
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,Fiber optic,No,...,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40,Yes,True
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,DSL,No,...,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),69.40,571.45,No,False
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),109.70,7904.25,No,True
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,Fiber optic,Yes,...,Yes,No,No,Two year,Yes,Credit card (automatic),84.65,5377.80,No,False
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,48.20,340.35,No,True
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Two year,Yes,Mailed check,90.45,5957.90,No,False


In [37]:
from pyspark.sql.functions import countDistinct

#Check schema & data types
silver_df.show(10, truncate=False)

#Quick sample of cleaned data

silver_df.show(10, truncate=False)

#Row count before & after cleaning

raw_count = bronze_df.count()
clean_count = silver_df.count()
print(f"Before cleaning: {raw_count}, After cleaning: {clean_count}")

#Deduplication check

total_customers = silver_df.count()
unique_customers = silver_df.select("customerID").distinct().count()
print(f"Total rows: {total_customers}, Unique customers: {unique_customers}")


#Data Quality Checks

#Tenure should not be negative
silver_df.filter(col("tenure") < 0).show()

#MonthlyCharges & TotalCharges should be non-negative
silver_df.filter((col("MonthlyCharges") < 0) | (col("TotalCharges") < 0)).show()

#SeniorCitizen → should only be 0/1
silver_df.groupBy("SeniorCitizen").count().show()


+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+-------------------------+--------------+------------+-----+--------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract      |PaperlessBilling|PaymentMethod            |MonthlyCharges|TotalCharges|Churn|IsSenior|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+-------------------------+--------------+------------+-----+--------+
|0002-ORFBO|Female|0            |Yes    |Yes       |9     |Yes         |No           |DSL            |No            |Yes         |No 

# GOLD LAYER
### HERE WE ARE USING STAR SCHEMA ARCHITECTURE 

In [28]:
#READ SILVER LAYER

silver_path = "/home/Personal_Projects/Databricks/work/data/silver/telco_churn"
silver_df = spark.read.parquet(silver_path)
silver_df.show(5, truncate=False)


+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+--------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract      |PaperlessBilling|PaymentMethod   |MonthlyCharges|TotalCharges|Churn|IsSenior|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+--------+
|0002-ORFBO|Female|0            |Yes    |Yes       |9     |Yes         |No           |DSL            |No            |Yes         |No              |Yes        |Y

In [6]:
#Create Dimension Tables
# Customer Dimension
dim_customer = silver_df.select(
    "customerID",
    "gender",
    "SeniorCitizen",
    "IsSenior",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines"
).dropDuplicates(["customerID"])

# Contract Dimension
dim_contract = silver_df.select(
    "Contract",
    "PaperlessBilling"
).dropDuplicates()

# Internet & Services Dimension
dim_services = silver_df.select(
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies"
).dropDuplicates()

# Payment Dimension
dim_payment = silver_df.select(
    "PaymentMethod"
).dropDuplicates()


In [7]:
#Create Fact Table
# Fact table includes metrics + foreign keys to dimensions
fact_churn = silver_df.select(
    "customerID",       # FK to Customer
    "Contract",         # FK to Contract
    "InternetService",  # FK to Services
    "PaymentMethod",    # FK to Payment
    "tenure",
    "MonthlyCharges",
    "TotalCharges"
)


In [8]:
#Save Gold Layer Tables as Parquet
gold_base_path = "/home/Personal_Projects/Databricks/work/data/gold/"

dim_customer.write.mode("overwrite").parquet(gold_base_path + "dim_customer")
dim_contract.write.mode("overwrite").parquet(gold_base_path + "dim_contract")
dim_services.write.mode("overwrite").parquet(gold_base_path + "dim_services")
dim_payment.write.mode("overwrite").parquet(gold_base_path + "dim_payment")
fact_churn.write.mode("overwrite").parquet(gold_base_path + "fact_churn")


In [9]:
#CHECKING THE TABLES
dim_customer.show(10)
fact_churn.show(10)
dim_payment.show(10)

+----------+------+-------------+--------+-------+----------+------------+-------------+
|customerID|gender|SeniorCitizen|IsSenior|Partner|Dependents|PhoneService|MultipleLines|
+----------+------+-------------+--------+-------+----------+------------+-------------+
|0002-ORFBO|Female|            0|   false|    Yes|       Yes|         Yes|           No|
|0003-MKNFE|  Male|            0|   false|     No|        No|         Yes|          Yes|
|0004-TLHLJ|  Male|            0|   false|     No|        No|         Yes|           No|
|0011-IGKFF|  Male|            1|    true|    Yes|        No|         Yes|           No|
|0013-EXCHZ|Female|            1|    true|    Yes|        No|         Yes|           No|
|0013-MHZWF|Female|            0|   false|     No|       Yes|         Yes|           No|
|0013-SMEOE|Female|            1|    true|    Yes|        No|         Yes|           No|
|0014-BMAQU|  Male|            0|   false|    Yes|        No|         Yes|          Yes|
|0015-UOCOJ|Female|  

In [11]:


# Convert all Gold tables to lowercase column names
dim_customer_lower = dim_customer.toDF(*[c.lower() for c in dim_customer.columns])
dim_contract_lower = dim_contract.toDF(*[c.lower() for c in dim_contract.columns])
dim_services_lower = dim_services.toDF(*[c.lower() for c in dim_services.columns])
dim_payment_lower = dim_payment.toDF(*[c.lower() for c in dim_payment.columns])
fact_churn_lower = fact_churn.toDF(*[c.lower() for c in fact_churn.columns])

# Write them out with lowercase schema
dim_customer_lower.write.mode("overwrite").parquet(gold_base_path + "dim_customer")
dim_contract_lower.write.mode("overwrite").parquet(gold_base_path + "dim_contract")
dim_services_lower.write.mode("overwrite").parquet(gold_base_path + "dim_services")
dim_payment_lower.write.mode("overwrite").parquet(gold_base_path + "dim_payment")
fact_churn_lower.write.mode("overwrite").parquet(gold_base_path + "fact_churn")
